In [0]:
import sys

sys.path.append("../include_utils/")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from hdfstorehelper import HDFStoreHelper
import dill
from scipy import stats
from IPython.display import display
import geopy

samtools = "/home/cfriedline/gpfs/src/samtools-1.3/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_60/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

vcfutils = "perl /home/cfriedline/g/src/bcftools-1.3/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
tabix = "/home/cfriedline/gpfs/src/htslib-1.3/tabix"
bgzip = "/home/cfriedline/gpfs/src/htslib-1.3/bgzip"


def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

setup_r()
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext rpy2.ipython

In [0]:
ni_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni"
imp_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40"
notimputed_vcf_gz = os.path.join(ni_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
imputed_vcf_gz = os.path.join(imp_dir, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
vcf_files = [notimputed_vcf_gz, imputed_vcf_gz]

In [0]:
hierf_trans = {0:11, 1:12, 2:22, -1:'NA'}
def apply_hierf_trans(series):
    return [hierf_trans[x] if x in hierf_trans else x for x in series]

In [0]:
def read_df(dirname, fname):
    f = os.path.join(dirname, "%s.txt" % fname)
    return pd.read_csv(f, sep="\t", index_col=0)

In [0]:
z12_swapped = [read_df(x, "z12_swapped") for x in [ni_dir, imp_dir]]

In [0]:
z12_swapped[0].head()

In [0]:
hierf_df = [x.ix[:,:-2].apply(apply_hierf_trans) for x in z12_swapped]

In [0]:
hierf_df[0].shape

In [0]:
for i, df in enumerate(hierf_df):
    hierf_df[i] = z12_swapped[i][['popid']].join(df)

In [0]:
for i, elem in enumerate(vcf_files):
    filedir = os.path.dirname(elem)
    outfile = os.path.join(filedir, "isect_hierfstat.txt")
    hierf_df[i].to_csv(outfile, header=True, index=False, sep="\t")

# Put into R (because it can be slow)

```
library(hierfstat)
library(data.table)
data = fread("isect_hierfstat.txt", header=T, sep="\t", data.table=F)
levels = data.frame(data$popid)
loci = data[,2:ncol(data)]
bs = basic.stats(data)
saveRDS(bs, "isect_hierfstat_basic_stats.rds")
res = varcomp.glob(levels=levels, loci=loci, diploid=T)
saveRDS(res, "isect_hierfstat_varcomp.rds")
```

### Also compute pairwise Fst (qrsh)
```
rm(list=ls())
library(hierfstat)
library(data.table)
library(snow)
data = fread("isect_hierfstat.txt", header=T, sep="\t", data.table=F)
levels = data.frame(data$popid)
loci = data[,2:ncol(data)]

run_varcomp = function(idx) {
    i = args[[idx]]$i
    j = args[[idx]]$j
    key = paste(i, j, sep="-")
    d = copy(data)
    d = subset(d, d$popid == i | d$popid == j)
    levels = data.frame(d$popid)
    loci = d[,2:ncol(d)]
    return(varcomp.glob(levels=levels, loci=loci, diploid=T))
}

args = list()
for (i in 1:6) {
    for (j in 1:i) {
        if (i != j) {
            args[[length(args)+1]] = list(i=i, j=j)
        }
    }
}

hosts = rep("localhost", length(args))
cl = makeSOCKcluster(hosts)
clusterExport(cl, "data")
clusterEvalQ(cl, library(hierfstat))
clusterEvalQ(cl, library(data.table))
clusterExport(cl, "args")
clusterExport(cl, "run_varcomp")
pairwise_res = parLapply(cl, 1:length(args), "run_varcomp")
saveRDS(pairwise_res, "isect_hierfstat_pairwise.rds")
saveRDS(args, "isect_hierfstat_pairwise_args.rds")
stopCluster(cl)

```

In [0]:
%%R
varcomp_not = readRDS("/gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni/isect_hierfstat_varcomp.rds")
bs_not = readRDS("/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/ni/isect_hierfstat_basic_stats.rds")
varcomp_imp = readRDS("/gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/isect_hierfstat_varcomp.rds")
bs_imp = readRDS("/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3//beagle40/isect_hierfstat_basic_stats.rds")

In [0]:
def get_r_series(key):
    s = pd.Series(get_r(key))
    s.index = get_r("names(%s)" % key)
    return s

def get_r_df(key):
    df = pd.DataFrame(get_r(key))
    try:
        rname = get_r("rownames(%s)" % key)
        df.index = rname
    except:
        pass
    
    try:
        cname = get_r("colnames(%s)" % key)
        df.columns = cname
    except:
        pass
    
    return df

def get_r(key):
    return r(key)

In [0]:
perloc_not = get_r_df("bs_not$perloc")
Ho_not = get_r_df("bs_not$Ho")
Hs_not = get_r_df("bs_not$Hs")
Fis_not = get_r_df("bs_not$Fis")
overall_not = get_r_series("bs_not$overall")
n_ind_samp_not = get_r_df("bs_not$n.ind.samp")

In [0]:
perloc_imp = get_r_df("bs_imp$perloc")
Ho_imp = get_r_df("bs_imp$Ho")
Hs_imp = get_r_df("bs_imp$Hs")
Fis_imp = get_r_df("bs_imp$Fis")
overall_imp = get_r_series("bs_imp$overall")
n_ind_samp_imp = get_r_df("bs_imp$n.ind.samp")

In [0]:
def save_df(dirname, fname, df):
    f = os.path.join(dirname, "%s.txt" % fname) 
    df.to_csv(f, 
              header=True,
              index=True,
              sep="\t")
    print("saved %s" % f)

In [0]:
loc_df_not = get_r_df('varcomp_not$loc')
F_df_not = get_r_df('varcomp_not$F')
overall_df_not = get_r_df('varcomp_not$overall')

In [0]:
loc_df_imp = get_r_df('varcomp_imp$loc')
F_df_imp = get_r_df('varcomp_imp$F')
overall_df_imp = get_r_df('varcomp_imp$overall')

In [0]:
loc_df_imp.head()

In [0]:
F_df_not

In [0]:
F_df_imp

In [0]:
def compute_fst(series):
    Hs = series[0]
    Ht = sum(series)
    return Hs/Ht

In [0]:
loci_fst_not = loc_df_not.apply(compute_fst, axis=1)
loci_fst_imp = loc_df_imp.apply(compute_fst, axis=1)

In [0]:
loci_fst_not.describe()

In [0]:
loci_fst_imp.describe()

In [0]:
for i, d in enumerate([ni_dir, imp_dir]):
    if i == 0:
        save_df(d, 'perloc', perloc_not)
        save_df(d, 'Ho', Ho_not)
        save_df(d, "Hs", Hs_not)
        save_df(d, 'Fis', Fis_not)
        save_df(d, 'overall', overall_not)
        save_df(d, 'n_ind_samp', n_ind_samp_not)
        save_df(d, 'loc_df', loc_df_not)
        save_df(d, 'F_df', F_df_not)
        save_df(d, 'varcomp_overall', overall_df_not)
        save_df(d, 'loci_fst', loci_fst_not)
    else:
        save_df(d, 'perloc', perloc_imp)
        save_df(d, 'Ho', Ho_imp)
        save_df(d, "Hs", Hs_imp)
        save_df(d, 'Fis', Fis_imp)
        save_df(d, 'overall', overall_imp)
        save_df(d, 'n_ind_samp', n_ind_samp_imp)
        save_df(d, 'loc_df', loc_df_imp)
        save_df(d, 'F_df', F_df_imp)
        save_df(d, 'varcomp_overall', overall_df_imp)
        save_df(d, 'loci_fst', loci_fst_imp)

In [0]:
plt.hist(loci_fst_not, bins=50)
plt.title("not imputed n=%d mean=%.4f +/- %.4f [%.4f, %.4f]" % (len(loci_fst_not), 
                                                    np.mean(loci_fst_not), 
                                                    np.std(loci_fst_not),
                                                    np.min(loci_fst_not), 
                                                    np.max(loci_fst_not)))
plt.xlabel(r"$F_{ST}$")
plt.show()

plt.hist(loci_fst_imp, bins=50)
plt.title("imputed n=%d mean=%.4f +/- %.4f [%.4f, %.4f]" % (len(loci_fst_imp), 
                                                    np.mean(loci_fst_imp), 
                                                    np.std(loci_fst_imp),
                                                    np.min(loci_fst_imp), 
                                                    np.max(loci_fst_imp)))
plt.xlabel(r"$F_{ST}$")
plt.show()

In [0]:
popid_map = {}
for population, data in z12_swapped[1].groupby("population"):
    popid = data['popid'].unique()[0]
    print(population, popid)
    popid_map[popid] = population

In [0]:
perloc = read_df(imp_dir, "perloc")
loc_df = read_df(imp_dir, "loc_df")
Ho = read_df(imp_dir, "Ho")
Ho.columns = [popid_map[int(x)] for x in Ho.columns]
overall = read_df(imp_dir, "overall")
n_ind_samp = read_df(imp_dir, "n_ind_samp")
n_ind_samp.columns = [[popid_map[int(x)] for x in n_ind_samp.columns]]
fis = read_df(imp_dir, "Fis")
fis.columns = [popid_map[int(x)] for x in fis.columns]

In [0]:
Ho.to_csv("Ho_labelled.txt", 
          index=True, 
          header=True, 
          sep="\t")

In [0]:
from IPython.display import FileLink
FileLink("Ho_labelled.txt")

In [0]:
Ho.head()

In [0]:
Ho.columns

In [0]:
stats.f_oneway(Ho['NC'], Ho['NY'], Ho['QC32'], Ho['QC93'], Ho['VA1'], Ho['VA2'])

In [0]:
sns.set_context("talk")

In [0]:
sns.boxplot(data=Ho);

In [0]:
%%R
bs_imp$pop.freq$ctg7180005039298_50

In [0]:
pop_freq = r('bs_imp$pop.freq')

pop_freq_names = pandas2ri.ri2py(pop_freq.names)

In [0]:
def compute_He(elem):
    He = 2
    for x in elem:
        He *= x
    return He

He_dict = {}

for i, name in enumerate(pop_freq_names):
    af = pandas2ri.ri2py_dataframe(pop_freq.rx2(name))
    af.columns = [x+1 for x in af.columns]
    He_dict[name] = af.apply(compute_He).to_dict()
    
    if i % 10000 == 0:
        print("at %d" % i)

In [0]:
He = pd.DataFrame(He_dict).T

In [0]:
He.columns = [popid_map[x] for x in He.columns]

In [0]:
Ho_He = Ho.join(He, lsuffix = "_Ho", rsuffix = "_He")

In [0]:
means = pd.DataFrame(Ho_He.apply(np.mean)).T

In [0]:
means

In [0]:
diffs = Ho-He

In [0]:
sns.boxplot(data=diffs)
plt.ylabel(r"$Ho - He$")

In [0]:
sns.boxplot(data=np.abs(diffs))
plt.ylabel(r"$|Ho-He|$")

In [0]:
%%R
pairwise = readRDS("/gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/isect_hierfstat_pairwise.rds")
pairwise_args = readRDS("/gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/isect_hierfstat_pairwise_args.rds")


In [0]:
%%R
pairwise[1]

In [0]:
pairwise = r("pairwise")

In [0]:
pairwise_fst = np.zeros((7,7))
for arg in range(len(pairwise_args)):
    arg = arg+1
    i = pairwise_args.rx2(arg).rx2("i")[0]
    j = pairwise_args.rx2(arg).rx2("j")[0]
    print(i, j)
    F = pandas2ri.ri2py_dataframe(pairwise.rx2(arg).rx2("F"))
    pairwise_fst[i, j] = F.ix[0,0]

In [0]:
pd.DataFrame(pairwise_fst)

In [0]:
pop_dict = {}
for idx, data in z12_swapped[1][['population', 'popid']].iterrows():
    pop_dict[data.popid] = data.population

In [0]:
np.max(pairwise_df.max())

In [0]:
pairwise_df = pd.DataFrame(pairwise_fst)
pairwise_df = pairwise_df.drop(0, axis=1).drop(6, axis=1).drop(0)
pairwise_df.columns = [pop_dict[x] for x in pairwise_df]
pairwise_df.index = [pop_dict[x] for x in pairwise_df.index]
pairwise_df.replace(0, False)

In [0]:
sns.set_context("talk")
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
sns.heatmap(pairwise_df, vmin=0, vmax=pairwise_df.max().max(), cmap=cmap, annot=True)
plt.title("Pairwise multilocus %s" % r'$F_{ST}$')
plt.show()

In [0]:
latlon = read_df(imp_dir, "bioclim_df")[["lat", "lon"]]

In [0]:
latlon

In [0]:
from geopy.distance import vincenty
dist = {}
for i in range(len(latlon.index)):
    ipop = latlon.index[i]
    icoord = (latlon.ix[ipop].lat, latlon.ix[ipop].lon)
    if not ipop in dist:
        dist[ipop] = {}
    for j in range(i):
        jpop = latlon.index[j]
        jcoord = (latlon.ix[jpop].lat, latlon.ix[jpop].lon)
        d = vincenty(icoord, jcoord).km
        print(ipop, jpop, d)
        dist[ipop][jpop] = d
        if not jpop in dist:
            dist[jpop] = {}
        dist[jpop][ipop] = d

In [0]:
dist_df = pd.DataFrame(dist)

In [0]:
fst_gps = {}
for p in pairwise_df.index:
    for q in pairwise_df.index:
        if not p == q:
            print(p, q)
            fst = -1
            try:
                fst = pairwise_df.ix[p, q]
            except:
                fst = pairwise_df.ix[q, p]
            key = "-".join(sorted([p, q]))
            fst_gps[key] = {"vincenty": dist_df.ix[p,q],
                                         "fst": fst}

In [0]:
fst_gps_df = pd.DataFrame(fst_gps).T

In [0]:
fst_gps_df

In [0]:
g = sns.lmplot("fst", "vincenty", data=fst_gps_df, scatter_kws={"s": 100})
g.fig.set_size_inches(10, 10)
for i in fst_gps_df.index:
    plt.annotate(i, 
                 xy=(fst_gps_df.ix[i, "fst"], fst_gps_df.ix[i, "vincenty"]), 
                xytext = (-10, 20),
                textcoords = 'offset points', ha = 'right', va = 'bottom', 
                arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
plt.ylabel("Vincenty (km)")
plt.xlabel(r'Pairwise $F_{ST}$')
plt.show()
#bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
#arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0')

In [0]:
from statsmodels.formula.api import ols

In [0]:
ols("fst~vincenty", data=fst_gps_df).fit().summary()